In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import napari
from skimage.measure import label
import cc3d
from tifffile import imread, imwrite
from glob import glob
from utils import *
from skimage.morphology import disk, binary_dilation
import matplotlib.pyplot as plt
import pandas


Project uses napari viewer for interaction with images. Installation of napari is necessary. 

In [3]:
# start the viewer
viewer = napari.Viewer()

Original images should be avi videos. If raw images are tiff file use next cell to load them.

In [3]:
# open original data and add it to napari viewer
filepath = 'video_sperm/Jeko1_videa/FcRL3_siRNAIII.avi'
image_data = get_image_data(filepath)
viewer.add_image(image_data, name='frames')

<Image layer 'frames' at 0x29b7a80d0>

In [4]:
# if raw images are tiff filees
path = 'hek ctrl J F/HEK ctrl 1.ome.tif'
raw = imread(path)

viewer.add_image(raw, name='raw')

<Image layer 'raw' at 0x2b1ccae20>

Load respective masks for the raw images. Masks should be time-stacks saved as tiff files.

In [7]:
# open segmentation
path = 'hek ctrl J F/HEK ctrl 1_cells.ome.tif'
cells = imread(path)

path = 'hek ctrl J F/HEK ctrl 1_sperm.ome.tif'
sperm = imread(path)

# label masks
sperm_labels = label(sperm)
cell_labels = label(cells)

# add both labels to napari viewer
viewer.add_labels(cell_labels, name='cells')
viewer.add_labels(sperm_labels, name='sperm')

<Labels layer 'sperm [1]' at 0x13043b8b0>

AGX: exceeded compiled variants footprint limit


To select which cell to analyze open viewer and add "points" layer, then add point over the cell of interest.

In [ ]:
# choose cell
point = viewer.layers['Points'].data

In [23]:
cells = viewer.layers['cells'].data
sperm = viewer.layers['sperm'].data

# take cells value from point layer
cell_id = cells[0][int(point[0][1])][int(point[0][2])]

# take only cell with cell_id
cell = cells.copy()
cell[cells != cell_id] = 0
viewer.add_labels(cell, name='cell')

<Labels layer 'cell' at 0x2aad67820>

In [24]:
# dilate cell by 3 pixels
cell_dilated = np.zeros_like(cell)
for i in range(cell.shape[0]):
    # dilate using cv2
    kernel = np.ones((3, 3), np.uint8)
    cell_dilated[i] = cv2.dilate(cell[i].astype(np.uint8), kernel=kernel, iterations=4)

viewer.add_labels(cell_dilated, name='cell_dilated')

<Labels layer 'cell_dilated' at 0x2aeaae850>

Here, we use dilated sperm cell to define the region of interaction between cell and sperm cell.

In [25]:
# if any sperm is touching cell dilated, then sperm is interacting with cell
sperm = viewer.layers['sperm'].data
sperm_interacting = np.multiply(sperm, cell_dilated)
print(np.unique(sperm_interacting))

[     0   8528  28700  39688 205984]


In [26]:
sperm_interacting_id, counts = np.unique(sperm_interacting, return_counts=True)
sperm_interacting_id = sperm_interacting_id[counts > 1000]
sperm_interacting_id = sperm_interacting_id[1:]
print(sperm_interacting_id, counts)

[8528] [637118099     10242        85        57         9]


Perform measurements through all time stacks and save results to csv table.

In [27]:
for k, interaction in enumerate(sperm_interacting_id):
    print('iteration: ', k)
    # if sperm and sperm interacting overlap, get this sperm value
    sperm_coords = np.array(np.where(sperm_interacting == interaction)).T[0]
    sperm = viewer.layers['sperm'].data
    sperm_cell_value = sperm[sperm_coords[0], sperm_coords[1], sperm_coords[2]]

    # create sperm cell 
    sperm_cell = sperm.copy()
    sperm_cell[sperm != sperm_cell_value] = 0
    viewer.add_labels(sperm_cell, name='sperm_cell')

    print('sperm_cell shape is', sperm_cell.shape, 'with labels', np.unique(sperm_cell))
    
    # create an empty array for angles values
    angles = []

    # now do angle measurements for all time frames
    cell = viewer.layers['cell_dilated'].data
    
    # find stack where the overlap is present
    overlap = sperm_interacting.copy()
    overlap[overlap != interaction] = 0

    for stack in overlap:
        if np.sum(overlap) > 0:
            # get overlap coordinates
            overlap_coords = np.array(np.where(stack != 0))
            break
    print('overlap_coords', overlap_coords)

    # compute cell line and cell vector
    cell_line, cell_vector = compute_cell_vector(overlap_coords)

    print('cell_line', cell_line)

    n = 2 * sperm_cell.shape[0]
    d = 3
    pos = np.zeros(shape=(n, 2, d), dtype=np.float32)
    vector_number = 0

    # lines = []
    for i, sperm in enumerate(sperm_cell):
        sperm_line, sperm_vector = compute_sperm_cell_vector(sperm)
        intersection = find_intersection(cell_line, sperm_line)

        if intersection == None:
            continue

        angle = compute_angle(cell_vector, sperm_vector)
        print(f'angle between cell and sperm on plane {i}: ', angle)
        angles.append(angle)

        # if i == 1:
        pos[vector_number, 0, :] = (i, *intersection)
        pos[vector_number, 1, :] = (i, *sperm_vector)
        vector_number += 1

        pos[vector_number, 0, :] = (i, *intersection)
        pos[vector_number, 1, :] = (i, *cell_vector)
        vector_number += 1
        
          
    vect = viewer.add_vectors(pos, edge_width=1, length=10, name='cell vect')

    # create dataframe
    df = pandas.DataFrame(angles)
    # rename column
    df.columns = ['angle']
    # add new column with same value
    df['time'] = np.arange(len(angles))
    # compute angle change
    df['angle_change'] = abs(df['angle'].diff())
    # save to csv
    df.to_csv(f'cell_{point[0][1], point[0][2]}_{k}.csv')
    print(df)
        

iteration:  0
sperm_cell shape is (203, 1534, 2046) with labels [ 0 52]
overlap_coords [[903 903 903 904 904 904 904 905 905 905 905 906 906 906 906 907 907 907
  907 907 907 908 908 908 908 908 909 909 909 909 910 910 910 910 910 911
  911 911 911 911 912 912 912 912 913 913 913 913 914 914 914 915 915 916
  917]
 [ 55  56  57  56  57  58  59  57  58  59  60  58  59  60  61  58  59  60
   61  62  63  59  60  61  62  63  60  61  62  63  60  61  62  63  64  61
   62  63  64  65  63  64  65  66  64  65  66  67  65  66  67  67  68  68
   69]]
cell_line (array([903,  55]), array([917,  69]))
angle between cell and sperm on plane 0:  28.66395711028153
angle between cell and sperm on plane 1:  28.300755766006375
angle between cell and sperm on plane 2:  27.181111085477212
angle between cell and sperm on plane 3:  27.52522574374433
angle between cell and sperm on plane 4:  20.43282867939867
angle between cell and sperm on plane 5:  25.346175941946676
angle between cell and sperm on plane 6:  